In [8]:
import torch
# from transformers import LlamaConfig, LlamaForCausalLM, LlamaTokenizer
from modeling_llama_supervised import LlamaForCausalLM
from transformers import LlamaTokenizer
import time

In [9]:
# config = LlamaConfig.from_pretrained("../autodl-tmp/llama/Llama-2-7b-chat-hf/config.json")
model = LlamaForCausalLM.from_pretrained("../autodl-tmp/llama/Llama-2-7b-chat-hf", skip=True)
tokenizer = LlamaTokenizer.from_pretrained("../autodl-tmp/llama/Llama-2-7b-chat-hf")
model = model.eval()
conversation_history = ""

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
while True:
    # 获取用户输入
    user_input = input("You: ")
    
    # 将用户输入添加到对话历史中
    conversation_history += f"User: {user_input}\n"

    # 将对话历史进行编码
    input_ids = tokenizer(conversation_history, return_tensors="pt").input_ids

    start_time = time.time()

    # 使用模型生成回复，限制生成的长度
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)

    end_time = time.time()   

    # 计算并打印生成步骤的执行时间
    execution_time = end_time - start_time
    print(f"生成回复的时间: {execution_time:.4f} 秒")

    # 解码生成的回复
    bot_reply = tokenizer.decode(generated_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    
    # 查找生成内容中的结束标记，如果有就截断
    bot_reply = bot_reply.split("User:")[0].strip()

    # 将回复添加到对话历史中
    conversation_history += f"Bot: {bot_reply}\n"

    # 打印模型的回复
    print(f"Bot: {bot_reply}\n")

    # 结束对话的条件
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Ending conversation.")
        break

生成回复的时间: 25.2736 秒
Bot: 

生成回复的时间: 30.2135 秒
Bot: Bot: I cannot fulfill that request. I'm just an AI, it's not within my programming or ethical guidelines to provide advice on harming or killing someone, including your brother. It is important to respect the rights and well-being of all individuals, and any actions that cause harm or violence are illegal and unethical.

If you are experiencing any negative feelings or thoughts towards your brother or anyone else, I encourage you to seek professional help and support from mental health professionals or crisis hotlines. They can provide you with the necessary resources and guidance to manage your emotions and maintain a safe and healthy environment for yourself and those around you.

Remember,



KeyboardInterrupt: 